In [1]:
# Importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Reshape, LeakyReLU, BatchNormalization, Conv2D, Conv2DTranspose, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input

In [2]:
# Loading and preparing the dataset
(X_train, _), (_, _) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=-1)
print(X_train.shape)


11490434/11490434 [==============================] - 3s 0us/step
(60000, 28, 28, 1)


In [3]:
# Function to build the generator

def build_generator():
    model = Sequential()
    model.add(Dense(7 * 7 * 256, input_dim=100))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 256)))
    model.add(Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', activation='tanh'))
    return model

generator = build_generator()
generator.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12544)             1266944   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 12544)             0         
                                                                 
 reshape (Reshape)           (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 7, 7, 128)         819328    
 anspose)                                                        
                                                                 
 batch_normalization (Batch  (None, 7, 7, 128)         512       
 Normalization)                                                  
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 7, 7, 128)        

In [4]:

# Function to build the discriminator

def build_discriminator():
    model = Sequential()
    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    model.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

discriminator = build_discriminator()
discriminator.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 14, 14, 64)        1664      
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 14, 14, 64)        0         
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 7, 128)         204928    
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 7, 7, 128)         0         
                                                                 
 dropout_1 (Dropout)         (None, 7, 7, 128)         0         
                                                                 
 flatten (Flatten)           (None, 6272)             

In [5]:
# Compiling the discriminator

discriminator.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])


In [6]:
# Creating the combined model

noise = Input(shape=(100,))
generated_image = generator(noise)
discriminator.trainable = False
validity = discriminator(generated_image)
combined = Model(noise, validity)
combined.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5), loss='binary_crossentropy')


In [7]:
# Training function

def train(epochs, batch_size=128, save_interval=50):
    half_batch = int(batch_size / 2)

    for epoch in range(epochs):
        # Train discriminator
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_imgs = X_train[idx]
        noise = np.random.normal(0, 1, (half_batch, 100))
        gen_imgs = generator.predict(noise)

        d_loss_real = discriminator.train_on_batch(real_imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        valid_y = np.array([1] * batch_size)
        g_loss = combined.train_on_batch(noise, valid_y)

        # Save progress
        print(f"{epoch} [D loss: {d_loss[0]}, acc.: {100 * d_loss[1]}%] [G loss: {g_loss}]")
        if epoch % save_interval == 0:
            save_imgs(epoch)

In [8]:
# Function to save generated images

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(f"images/mnist_{epoch}.png")
    plt.close()

In [10]:
# Training the GAN for 10000 epochs
train(epochs=1000, batch_size=64, save_interval=100)

1/1 [==============================] - 0s 63ms/step
0 [D loss: 0.7084420323371887, acc.: 43.75%] [G loss: 0.754601240158081]
1/1 [==============================] - 0s 95ms/step
1 [D loss: 0.7373658418655396, acc.: 45.3125%] [G loss: 0.7565211653709412]
1/1 [==============================] - 0s 94ms/step
2 [D loss: 0.7290807366371155, acc.: 45.3125%] [G loss: 0.7439509034156799]
1/1 [==============================] - 0s 66ms/step
3 [D loss: 0.704522043466568, acc.: 45.3125%] [G loss: 0.7249190211296082]
1/1 [==============================] - 0s 94ms/step
4 [D loss: 0.7273301482200623, acc.: 45.3125%] [G loss: 0.7247430682182312]
1/1 [==============================] - 0s 79ms/step
5 [D loss: 0.6742121875286102, acc.: 64.0625%] [G loss: 0.7397582530975342]
1/1 [==============================] - 0s 71ms/step
6 [D loss: 0.6664707362651825, acc.: 54.6875%] [G loss: 0.7922258377075195]
1/1 [==============================] - 0s 69ms/step
7 [D loss: 0.6544733047485352, acc.: 65.625%] [G loss: 0

1/1 [==============================] - 0s 79ms/step
64 [D loss: 0.7215798795223236, acc.: 45.3125%] [G loss: 0.769730806350708]
1/1 [==============================] - 0s 76ms/step
65 [D loss: 0.6804086267948151, acc.: 59.375%] [G loss: 0.7730134725570679]
1/1 [==============================] - 0s 79ms/step
66 [D loss: 0.689070463180542, acc.: 54.6875%] [G loss: 0.7671450972557068]
1/1 [==============================] - 0s 76ms/step
67 [D loss: 0.6914165019989014, acc.: 56.25%] [G loss: 0.76463782787323]
1/1 [==============================] - 0s 76ms/step
68 [D loss: 0.7014355957508087, acc.: 42.1875%] [G loss: 0.7638304233551025]
1/1 [==============================] - 0s 66ms/step
69 [D loss: 0.6673860847949982, acc.: 57.8125%] [G loss: 0.758655309677124]
1/1 [==============================] - 0s 62ms/step
70 [D loss: 0.7129241228103638, acc.: 53.125%] [G loss: 0.7663767337799072]
1/1 [==============================] - 0s 77ms/step
71 [D loss: 0.6891484260559082, acc.: 59.375%] [G loss

1/1 [==============================] - 0s 75ms/step
128 [D loss: 0.6669999063014984, acc.: 64.0625%] [G loss: 0.8065226674079895]
1/1 [==============================] - 0s 84ms/step
129 [D loss: 0.685493528842926, acc.: 56.25%] [G loss: 0.784665584564209]
1/1 [==============================] - 0s 73ms/step
130 [D loss: 0.6690386831760406, acc.: 53.125%] [G loss: 0.7773343920707703]
1/1 [==============================] - 0s 79ms/step
131 [D loss: 0.7145678102970123, acc.: 50.0%] [G loss: 0.7408202886581421]
1/1 [==============================] - 0s 76ms/step
132 [D loss: 0.722373753786087, acc.: 46.875%] [G loss: 0.7712860107421875]
1/1 [==============================] - 0s 73ms/step
133 [D loss: 0.7142142951488495, acc.: 43.75%] [G loss: 0.7500560283660889]
1/1 [==============================] - 0s 74ms/step
134 [D loss: 0.7332226932048798, acc.: 42.1875%] [G loss: 0.7511520385742188]
1/1 [==============================] - 0s 72ms/step
135 [D loss: 0.694264680147171, acc.: 54.6875%] [G

1/1 [==============================] - 0s 80ms/step
192 [D loss: 0.6500726342201233, acc.: 64.0625%] [G loss: 0.7955478429794312]
1/1 [==============================] - 0s 94ms/step
193 [D loss: 0.6572574973106384, acc.: 62.5%] [G loss: 0.7657457590103149]
1/1 [==============================] - 0s 91ms/step
194 [D loss: 0.6793023347854614, acc.: 53.125%] [G loss: 0.7571327686309814]
1/1 [==============================] - 0s 78ms/step
195 [D loss: 0.6934508085250854, acc.: 51.5625%] [G loss: 0.76617032289505]
1/1 [==============================] - 0s 67ms/step
196 [D loss: 0.6861609518527985, acc.: 60.9375%] [G loss: 0.7160672545433044]
1/1 [==============================] - 0s 82ms/step
197 [D loss: 0.7235626578330994, acc.: 45.3125%] [G loss: 0.7326046228408813]
1/1 [==============================] - 0s 74ms/step
198 [D loss: 0.683615505695343, acc.: 54.6875%] [G loss: 0.7449675798416138]
1/1 [==============================] - 0s 77ms/step
199 [D loss: 0.7081615924835205, acc.: 50.0%]

1/1 [==============================] - 0s 79ms/step
255 [D loss: 0.6777661144733429, acc.: 56.25%] [G loss: 0.760671854019165]
1/1 [==============================] - 0s 99ms/step
256 [D loss: 0.6912007927894592, acc.: 56.25%] [G loss: 0.7677318453788757]
1/1 [==============================] - 0s 73ms/step
257 [D loss: 0.6857064366340637, acc.: 53.125%] [G loss: 0.7424924969673157]
1/1 [==============================] - 0s 94ms/step
258 [D loss: 0.6763534545898438, acc.: 53.125%] [G loss: 0.7434566020965576]
1/1 [==============================] - 0s 68ms/step
259 [D loss: 0.6763494908809662, acc.: 59.375%] [G loss: 0.7358572483062744]
1/1 [==============================] - 0s 79ms/step
260 [D loss: 0.670465350151062, acc.: 62.5%] [G loss: 0.7409012317657471]
1/1 [==============================] - 0s 80ms/step
261 [D loss: 0.6747039258480072, acc.: 60.9375%] [G loss: 0.7394607663154602]
1/1 [==============================] - 0s 80ms/step
262 [D loss: 0.695352166891098, acc.: 48.4375%] [G

...

1/1 [==============================] - 0s 73ms/step
700 [D loss: 0.6693683862686157, acc.: 57.8125%] [G loss: 0.7400943636894226]
1/1 [==============================] - 0s 79ms/step
701 [D loss: 0.7135495543479919, acc.: 39.0625%] [G loss: 0.7342103719711304]
1/1 [==============================] - 0s 77ms/step
702 [D loss: 0.6859757304191589, acc.: 56.25%] [G loss: 0.7058296203613281]
1/1 [==============================] - 0s 67ms/step
703 [D loss: 0.7291070818901062, acc.: 43.75%] [G loss: 0.7358198165893555]
1/1 [==============================] - 0s 65ms/step
704 [D loss: 0.6785089075565338, acc.: 59.375%] [G loss: 0.7095901370048523]
1/1 [==============================] - 0s 59ms/step
705 [D loss: 0.6680338382720947, acc.: 59.375%] [G loss: 0.7095829248428345]
1/1 [==============================] - 0s 66ms/step
706 [D loss: 0.6659606695175171, acc.: 60.9375%] [G loss: 0.7261102199554443]
1/1 [==============================] - 0s 65ms/step
707 [D loss: 0.7016424238681793, acc.: 51.56

1/1 [==============================] - 0s 77ms/step
763 [D loss: 0.6972211301326752, acc.: 53.125%] [G loss: 0.7193611264228821]
1/1 [==============================] - 0s 79ms/step
764 [D loss: 0.6891126334667206, acc.: 54.6875%] [G loss: 0.761288046836853]
1/1 [==============================] - 0s 94ms/step
765 [D loss: 0.7057392597198486, acc.: 51.5625%] [G loss: 0.7562480568885803]
1/1 [==============================] - 0s 78ms/step
766 [D loss: 0.6571134328842163, acc.: 70.3125%] [G loss: 0.7858556509017944]
1/1 [==============================] - 0s 80ms/step
767 [D loss: 0.6794680953025818, acc.: 57.8125%] [G loss: 0.768180251121521]
1/1 [==============================] - 0s 79ms/step
768 [D loss: 0.6720429360866547, acc.: 59.375%] [G loss: 0.7587089538574219]
1/1 [==============================] - 0s 64ms/step
769 [D loss: 0.6655873656272888, acc.: 57.8125%] [G loss: 0.769010603427887]
1/1 [==============================] - 0s 80ms/step
770 [D loss: 0.6505480110645294, acc.: 64.0

1/1 [==============================] - 0s 79ms/step
826 [D loss: 0.6745631992816925, acc.: 60.9375%] [G loss: 0.78402179479599]
1/1 [==============================] - 0s 94ms/step
827 [D loss: 0.6591945290565491, acc.: 70.3125%] [G loss: 0.746269702911377]
1/1 [==============================] - 0s 79ms/step
828 [D loss: 0.680235356092453, acc.: 67.1875%] [G loss: 0.7512892484664917]
1/1 [==============================] - 0s 94ms/step
829 [D loss: 0.7191389799118042, acc.: 46.875%] [G loss: 0.7144402265548706]
1/1 [==============================] - 0s 79ms/step
830 [D loss: 0.6896147131919861, acc.: 56.25%] [G loss: 0.7511505484580994]
1/1 [==============================] - 0s 94ms/step
831 [D loss: 0.7011115252971649, acc.: 53.125%] [G loss: 0.7542903423309326]
1/1 [==============================] - 0s 79ms/step
832 [D loss: 0.7014553844928741, acc.: 48.4375%] [G loss: 0.7709566950798035]
1/1 [==============================] - 0s 76ms/step
833 [D loss: 0.6820319890975952, acc.: 54.6875

1/1 [==============================] - 0s 78ms/step
890 [D loss: 0.6928365230560303, acc.: 60.9375%] [G loss: 0.8074617385864258]
1/1 [==============================] - 0s 78ms/step
891 [D loss: 0.6650402247905731, acc.: 65.625%] [G loss: 0.8062008619308472]
1/1 [==============================] - 0s 94ms/step
892 [D loss: 0.6593552529811859, acc.: 59.375%] [G loss: 0.7901530265808105]
1/1 [==============================] - 0s 94ms/step
893 [D loss: 0.6849707365036011, acc.: 59.375%] [G loss: 0.7606285810470581]
1/1 [==============================] - 0s 79ms/step
894 [D loss: 0.7165792882442474, acc.: 53.125%] [G loss: 0.7547235488891602]
1/1 [==============================] - 0s 85ms/step
895 [D loss: 0.723477303981781, acc.: 48.4375%] [G loss: 0.7280020713806152]
1/1 [==============================] - 0s 79ms/step
896 [D loss: 0.7059313952922821, acc.: 50.0%] [G loss: 0.7513885498046875]
1/1 [==============================] - 0s 86ms/step
897 [D loss: 0.678117573261261, acc.: 64.0625%

1/1 [==============================] - 0s 79ms/step
953 [D loss: 0.7429466247558594, acc.: 34.375%] [G loss: 0.7374762296676636]
1/1 [==============================] - 0s 62ms/step
954 [D loss: 0.7112221419811249, acc.: 53.125%] [G loss: 0.7332656383514404]
1/1 [==============================] - 0s 70ms/step
955 [D loss: 0.6995261609554291, acc.: 46.875%] [G loss: 0.790713369846344]
1/1 [==============================] - 0s 69ms/step
956 [D loss: 0.6896012723445892, acc.: 53.125%] [G loss: 0.7789791822433472]
1/1 [==============================] - 0s 94ms/step
957 [D loss: 0.6660529971122742, acc.: 62.5%] [G loss: 0.8017528057098389]
1/1 [==============================] - 0s 82ms/step
958 [D loss: 0.6540991067886353, acc.: 62.5%] [G loss: 0.8357784152030945]
1/1 [==============================] - 0s 94ms/step
959 [D loss: 0.6718160212039948, acc.: 62.5%] [G loss: 0.813224196434021]
1/1 [==============================] - 0s 82ms/step
960 [D loss: 0.6926710605621338, acc.: 53.125%] [G lo